# Load and preprocess data

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_txt = train_df['text'].to_list()
labels = train_df['target'].to_list()

In [4]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_txt, labels, test_size=.2)

## Import tokenizer from huggingface

In [5]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/home/jianant/.conda/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

## Prepare pytorch datasets 

In [7]:
import torch

class DisasterTwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DisasterTwitterDataset(train_encodings, train_labels)
val_dataset = DisasterTwitterDataset(val_encodings, val_labels)

# Finetune a pretrained model

## Prepare metric functions

In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Train the model

In [56]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,             # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    evaluation_strategy="epoch"      # evaluate the model after each epoch
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics      # metric computing function
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds"

Epoch,Training Loss,Validation Loss,Accuracy
1,0.497200,0.395401,0.840446
2,0.425100,0.381225,0.843729
3,0.210400,0.492221,0.830598
4,0.142600,0.705494,0.821405
5,0.117700,0.845637,0.831911
6,0.080300,0.862569,0.832567
7,0.040800,1.126820,0.827971
8,0.014400,1.195919,0.818122
9,0.012600,1.224659,0.837163
10,0.041300,1.252714,0.827971


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/con

TrainOutput(global_step=3810, training_loss=0.15569885907073816, metrics={'train_runtime': 217.684, 'train_samples_per_second': 279.763, 'train_steps_per_second': 17.502, 'total_flos': 1323535594852800.0, 'train_loss': 0.15569885907073816, 'epoch': 10.0})

# Make predictions on test data

## Load test data

In [19]:
test_df = pd.read_csv('test.csv')
test_txt = test_df['text'].to_list()

## Inference on test data using HuggingFace pipeline

In [57]:
from transformers import TextClassificationPipeline
pipeline =  TextClassificationPipeline(model = model, tokenizer=tokenizer, device=0, binary_output=True, return_all_scores=True)
result = pipeline(test_txt)

/home/jianant/.conda/envs/pytorch/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


## Transform the result into logits

In [58]:
'''The raw output has the format like this: [[{'label': 0, 'score':0.2}
                                              {'label': 1, 'score':0.8}],
                                             [{'label': 0, 'score':0.7}
                                              {'label': 1, 'score':0.3}],
                                              ...]
'''
pred = []
for r in result:
    if r[0]['score'] > r[1]['score']:
        pred.append(0)
    else:
        pred.append(1)

TypeError: '>' not supported between instances of 'dict' and 'dict'

## Save the results

In [ ]:
test_df['target'] = pred
test_df.drop(columns=['keyword', 'location', 'text'], inplace=True)
test_df.to_csv('submission.csv', index = False)

# Hyperparameter searching

## Define the parameters

In [11]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

## Initialize the model and trainer

In [13]:
def model_init():
    return DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [15]:
trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /home/jianant/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b6

## Search for the best hyperparameters (using opatune)

In [16]:
best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2022-08-25 20:16:58,384] A new study created in memory with name: no-name-d5ed13ad-809a-4311-ae11-97e5120e37a0
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/

Epoch,Training Loss,Validation Loss,Accuracy
1,0.445400,0.377138,0.842416


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-0/checkpoint-762
Configuration saved in ./results/run-0/checkpoint-762/config.json
Model weights saved in ./results/run-0/checkpoint-762/pytorch_model.bin
tokenizer config file saved in ./results/run-0/checkpoint-762/tokenizer_config.json
Special tokens file saved in ./results/run-0/checkpoint-762/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/run-0/checkpoint-762 (score: 0.8424162836506894).
[I 2022-08-25 20:17:31,942] Trial 0 finished with value: 0.8424162836506894 and parameters: {'learning_rate': 2.8124626544828694e-05, 'num_train_epochs': 1, 'seed': 32, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.8424162836506894.
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/hu

Epoch,Training Loss,Validation Loss,Accuracy
1,0.606400,0.492480,0.794485


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-1/checkpoint-762
Configuration saved in ./results/run-1/checkpoint-762/config.json
Model weights saved in ./results/run-1/checkpoint-762/pytorch_model.bin
tokenizer config file saved in ./results/run-1/checkpoint-762/tokenizer_config.json
Special tokens file saved in ./results/run-1/checkpoint-762/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/run-1/checkpoint-762 (score: 0.7944845699277742).
[I 2022-08-25 20:18:04,315] Trial 1 finished with value: 0.7944845699277742 and parameters: {'learning_rate': 1.778434201056688e-06, 'num_train_epochs': 1, 'seed': 31, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.8424162836506894.
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/hug

...

***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-7/checkpoint-762
Configuration saved in ./results/run-7/checkpoint-762/config.json
Model weights saved in ./results/run-7/checkpoint-762/pytorch_model.bin
tokenizer config file saved in ./results/run-7/checkpoint-762/tokenizer_config.json
Special tokens file saved in ./results/run-7/checkpoint-762/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/run-7/checkpoint-762 (score: 0.8411030860144452).
[I 2022-08-25 20:24:48,634] Trial 7 finished with value: 0.8411030860144452 and parameters: {'learning_rate': 1.508309136997296e-05, 'num_train_epochs': 1, 'seed': 31, 'per_device_train_batch_size': 8}. Best is trial 3 with value: 0.8483256730137886.
Trial:
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/hugg

Epoch,Training Loss,Validation Loss,Accuracy
1,0.473700,0.400871,0.842416
2,0.324800,0.502444,0.841760
3,0.238800,0.615344,0.838477
4,0.155000,0.736089,0.833880


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-8/checkpoint-762
Configuration saved in ./results/run-8/checkpoint-762/config.json
Model weights saved in ./results/run-8/checkpoint-762/pytorch_model.bin
tokenizer config file saved in ./results/run-8/checkpoint-762/tokenizer_config.json
Special tokens file saved in ./results/run-8/checkpoint-762/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-8/checkpoint-1524
Configuration saved in ./results/run-8/checkpoint-1524/config.json
Model weights saved in ./results/run-8/checkpoint-1524/pytorch_model.bin
tokenizer config file saved in ./results/run-8/checkpoint-1524/tokenizer_config.json
Special tokens file saved in ./results/run-8/checkpoint-1524/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-8/checkpoint-228

Epoch,Training Loss,Validation Loss,Accuracy
1,0.466700,0.387854,0.846356


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/run-9/checkpoint-762
Configuration saved in ./results/run-9/checkpoint-762/config.json
Model weights saved in ./results/run-9/checkpoint-762/pytorch_model.bin
tokenizer config file saved in ./results/run-9/checkpoint-762/tokenizer_config.json
Special tokens file saved in ./results/run-9/checkpoint-762/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/run-9/checkpoint-762 (score: 0.8463558765594222).
[I 2022-08-25 20:27:18,626] Trial 9 finished with value: 0.8463558765594222 and parameters: {'learning_rate': 1.2665158936229752e-05, 'num_train_epochs': 1, 'seed': 5, 'per_device_train_batch_size': 64}. Best is trial 3 with value: 0.8483256730137886.


## Use the best hyperparameters to train the model

In [17]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/jianant/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /home/jianant/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b6

Epoch,Training Loss,Validation Loss,Accuracy
1,0.492000,0.405781,0.839790
2,0.377900,0.395245,0.843729
3,0.345900,0.425233,0.837820
4,0.336400,0.443488,0.850295
5,0.314000,0.449801,0.848326


***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-762
Configuration saved in ./results/checkpoint-762/config.json
Model weights saved in ./results/checkpoint-762/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-762/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-762/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1524
Configuration saved in ./results/checkpoint-1524/config.json
Model weights saved in ./results/checkpoint-1524/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1524/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1524/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1523
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-2286
Configuration saved in ./results/checkpoint-2286/config.json
Mod

TrainOutput(global_step=3810, training_loss=0.367425052512662, metrics={'train_runtime': 153.0748, 'train_samples_per_second': 198.922, 'train_steps_per_second': 24.89, 'total_flos': 661767797426400.0, 'train_loss': 0.367425052512662, 'epoch': 5.0})

In [20]:
test_df = pd.read_csv('test.csv')
test_txt = test_df['text'].to_list()
from transformers import TextClassificationPipeline
pipeline =  TextClassificationPipeline(model = model, tokenizer=tokenizer, device=0, binary_output=True, return_all_scores=True)
result = pipeline(test_txt)
pred = []
for r in result:
    if r[0]['score'] > r[1]['score']:
        pred.append(0)
    else:
        pred.append(1)

/home/jianant/.conda/envs/pytorch/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(
Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [21]:
test_df['target'] = pred
test_df.drop(columns=['keyword', 'location', 'text'], inplace=True)
test_df.to_csv('submission.csv', index = False)